GLT CLASSIFIER

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, Subset
from torchvision import datasets, transforms, models
from torchvision.models import convnext_base, ConvNeXt_Base_Weights
from sklearn.model_selection import train_test_split
from PIL import Image
import numpy as np
import random

from tqdm import tqdm
import time

In [2]:
# Data augmentation and normalization for training
transform_train = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.25)
])

transform_val = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [3]:
# Load the full dataset
dataset_dir = '/content/drive/MyDrive/MLProjectColab/Reply_dataset'  # Adjust this path as per your folder structure
os.listdir(dataset_dir)
full_dataset = datasets.ImageFolder(root=dataset_dir)

In [4]:
# Split the dataset into train and validation sets
train_idx, val_idx = train_test_split(list(range(len(full_dataset))), test_size=0.2, stratify=full_dataset.targets)

train_dataset = Subset(full_dataset, train_idx)
val_dataset = Subset(full_dataset, val_idx)

# Apply transforms
train_dataset.dataset.transform = transform_train
val_dataset.dataset.transform = transform_val

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [5]:
# Define the AttentionPooling class
class AttentionPooling(nn.Module):
    def __init__(self, embed_dim):
        super(AttentionPooling, self).__init__()
        self.query = nn.Parameter(torch.randn(embed_dim, 1))  # Adjust to be a column vector

    def forward(self, x):
        # x shape: (batch_size, embed_dim, seq_len)
        attn_weights = torch.softmax(torch.matmul(x, self.query), dim=-2)  # (batch_size, seq_len, 1)
        attn_output = torch.matmul(x.transpose(1, 2), attn_weights).squeeze(-1)  # (batch_size, embed_dim)  # (batch_size, embed_dim)
        return attn_output

# Define the DocXClassifier model
class DocXClassifier(nn.Module):
    def __init__(self, num_classes=10):  # Set to 5 classes
        super(DocXClassifier, self).__init__()
        self.backbone = models.convnext_base(weights=ConvNeXt_Base_Weights.IMAGENET1K_V1)
        self.backbone = nn.Sequential(*list(self.backbone.children())[:-2])  # Keep only convolutional layers
        self.conv1x1 = nn.Conv2d(1024, 768, kernel_size=1)  # Point-wise convolution to transform dimensions
        self.pool = AttentionPooling(768)  # Adjust the dimension based on the ConvNeXt base
        self.classifier = nn.Linear(768, num_classes)

    def forward(self, x):
        features = self.backbone(x)  # Expected (batch_size, embed_dim, h, w)
        #print("Backbone output shape:", features.shape)
        features = self.conv1x1(features)  # Transform dimensions to (batch_size, 768, h, w)
        #print("After conv1x1 shape:", features.shape)
        features = features.flatten(2).transpose(1, 2)  # (batch_size, seq_len, embed_dim)
        #print("Flattened and transposed shape:", features.shape)
        pooled_features = self.pool(features)  # (batch_size, embed_dim)
        logits = self.classifier(pooled_features)  # (batch_size, num_classes)
        return logits

In [6]:
# Initialize model, loss, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DocXClassifier(num_classes=10).to(device)  # Tobacco3482 has 10 classes
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=8e-4)

In [8]:
from tqdm import tqdm

# Training loop with logging and tqdm progress bar
model.train()
num_epochs = 10  # example epoch count

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")

    # Training Phase
    model.train()
    train_loss = 0.0
    for batch_idx, (inputs, targets) in enumerate(tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch')):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        if batch_idx % 10 == 0:
            tqdm.write(f"Batch {batch_idx}, Loss: {loss.item():.4f}")

    # Calculate average training loss
    avg_train_loss = train_loss / len(train_loader)

    # Validation Phase
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in tqdm(val_loader, desc=f'Validation {epoch+1}/{num_epochs}', unit='batch'):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()

            # Calculate accuracy
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    # Calculate average validation loss and accuracy
    avg_val_loss = val_loss / len(val_loader)
    accuracy = 100. * correct / total

    # Print statistics
    print(f"Epoch {epoch+1}/{num_epochs} Training Loss: {avg_train_loss:.4f} Validation Loss: {avg_val_loss:.4f} Accuracy: {accuracy:.2f}%")



Epoch 1/10


Epoch 1/10:   1%|          | 1/123 [00:04<08:44,  4.30s/batch]

Batch 0, Loss: 2.2952


Epoch 1/10:   9%|▉         | 11/123 [00:49<08:01,  4.30s/batch]

Batch 10, Loss: 1.5058


...

Batch 110, Loss: 1.7165


Epoch 10/10:  98%|█████████▊| 121/123 [07:14<00:07,  3.58s/batch]

Batch 120, Loss: 0.8809


Validation 10/10: 100%|██████████| 31/31 [00:22<00:00,  1.36batch/s]

Epoch 10/10 Training Loss: 1.4060 Validation Loss: 1.3437 Accuracy: 55.51%
